In [1]:
import cv2
import numpy as np
import gradio as gr
from skimage import feature
from PIL import Image
from joblib import load
import dlib
from sklearn.preprocessing import MinMaxScaler,StandardScaler

# Load the pre-trained model
model = load("Model_DT.h5")

# Initialize the dlib face detector
detector = dlib.get_frontal_face_detector()
scaler = MinMaxScaler()

emotion_labels = {
    0: "Angry",
    1: "Fear",
    2: "Happy",
    3: "Neutral",
    4: "Sad",
}

def preprocess_image(image):
    """ Convert image to grayscale, resize, and compute HOG features. """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (48, 48))
    hog_features = feature.hog(resized, orientations=9, pixels_per_cell=(8, 8),
                               cells_per_block=(2, 2), block_norm='L2-Hys',
                               visualize=False, transform_sqrt=True)
    return hog_features

def predict(image):
    """ Detect faces, draw rectangles, preprocess image, and predict emotion. """
    if isinstance(image, Image.Image):
        image = np.array(image)

    # Detect faces
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    faces = detector(gray)

    if faces:
        face = faces[0]  # Focus on the first detected face
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        face_region = image[y:y+h, x:x+w]
        features = preprocess_image(face_region)
        features = features.reshape(1,-1)
        x = features.reshape(1,-1)
        x_mean = np.mean(x, axis=1, keepdims=True)
        x_std = np.std(x, axis=1, keepdims=True)
        x_scaled = (x - x_mean) / x_std
        print(x_scaled)
        prediction = model.predict(x_scaled)
        predicted_emotion = emotion_labels[int(prediction[0])]

        # Preparing the face image for display
        output_face_image = Image.fromarray(face_region)  # Convert NumPy array to PIL Image for Gradio
    else:
        predicted_emotion = "No face detected"
        output_face_image = Image.fromarray(image)  # Return the original image if no face detected

    return predicted_emotion, output_face_image

# Setup Gradio interface
iface = gr.Interface(
    fn=predict,
    inputs="image",
    outputs=["text", "image"],
    title="Emotion Detection",
    description="Upload an image to detect emotions in faces."
)

iface.launch(share = True)

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 4.14.0, however version 4.29.0 is available, please upgrade.
--------

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


[[-8.03114057e-01 -1.43164366e+00 -1.39646095e+00 -1.46035641e+00
  -1.17078746e+00 -1.41259317e+00 -1.15432243e+00 -7.62231844e-01
  -8.18200367e-01 -1.08735705e+00 -1.31962029e+00 -1.27867078e+00
  -1.24091344e+00 -1.19412339e+00 -1.06442056e+00 -1.24917021e+00
  -1.45749179e+00 -1.38014085e+00  5.02006434e-01 -9.04003989e-01
   1.73183382e+00  1.73183382e+00  1.73183382e+00  9.21137291e-01
   3.46124738e-01 -4.90644965e-01 -1.71865924e-01  2.68431096e-01
   1.06392628e+00  2.03778832e-01  1.73183382e+00  1.73183382e+00
   1.73183382e+00  1.73183382e+00  1.01938402e+00  9.77017471e-01
  -1.07343038e+00 -1.31346020e+00 -1.27114139e+00 -1.23212149e+00
  -1.18376685e+00 -1.04972692e+00 -1.24065436e+00 -1.45594194e+00
  -1.37600448e+00 -6.85402530e-01 -1.05470823e+00  1.62592730e+00
  -7.09846656e-01 -7.57269853e-01 -1.21606377e+00 -1.29467660e+00
  -1.30725069e+00 -1.38779468e+00  3.27693558e-01  1.14978907e+00
   2.60879406e-01  1.62592730e+00  1.62592730e+00  1.62592730e+00
   1.62592